In [ ]:

import numpy as np
import pandas as pd
import tensorflow

from keras.models import Sequential, Model
from keras.layers import Dense, Input, SimpleRNN, LSTM ,GRU
from sklearn.preprocessing import MinMaxScaler

!pip install -U finance-datareader
import FinanceDataReader as fdr


# 인풋 x 데이터 - 30개의 종가, 거래량 --> (30,2) 
# 라벨 y 데이터 - 5개의 종가, 거래량 --> (5,2)
def make_train_data(data):
  input_x_time_seq = 30
  output_y_time_seq = 5

  x_train = []
  y_train_close = []
  y_train_vol = []

  total_month = round(len(data) / 30)
  x_train_idx = 0

  for i in range(total_month):
    x_train_temp = data[x_train_idx:x_train_idx + input_x_time_seq] # data[0~29], data[30~59], data[60~89] ...
    x_train.append(x_train_temp) 
    x_train_idx += input_x_time_seq

    y_train_close_temp = data[x_train_idx: x_train_idx + output_y_time_seq, 0]  #data[30~35], data[60~65], data[90~95] ...
    y_train_vol_temp = data[x_train_idx: x_train_idx + output_y_time_seq, 1]  #data[30~35], data[60~65], data[90~95] ...
    y_train_close.append(y_train_close_temp)
    y_train_vol.append(y_train_vol_temp)
  
  return np.array(x_train), np.array(y_train_close), np.array(y_train_vol)

def normalize2(data):
  scaler_close = MinMaxScaler()
  scaler_vol = MinMaxScaler()
  norm_close = scaler_close.fit_transform(data[:, 0].reshape(-1, 1))
  norm_vol = scaler_vol.fit_transform(data[:, 1].reshape(-1, 1))
  norm_concated = np.hstack((norm_close, norm_vol))
  return norm_close, norm_vol, norm_concated, scaler_close, scaler_vol


def split_train_test(x_data, y_close, y_vol):
  train_num = round(len(x_data)*0.8)
  x_train, x_test = x_data[:train_num], x_data[train_num:]
  y_train_close, y_test_close = y_close[:train_num], y_close[train_num:]
  y_train_vol, y_test_vol = y_vol[:train_num], y_vol[train_num:]

  return x_train, x_test, y_train_close, y_test_close, y_train_vol, y_test_vol

# ref : https://gooopy.tistory.com/104, https://rinha7.github.io/multi_input_structure/
def ModelLearning(x_train, y_close, y_train, _epochs, _batch_size):
  input_layer = Input(shape=(30, 2))
  lstm_layer = LSTM(32)(input_layer)
  dense_hidden_layer = Dense(64, name='hidden', activation='relu')(lstm_layer) 
  dense_layer_close = Dense(5, name='close')(dense_hidden_layer)  # 종가 예측 5개
  dense_layer_volume = Dense(5, name='volume')(dense_hidden_layer)  # 거래량 예측 5개

  model = Model(inputs=input_layer, outputs=[dense_layer_close, dense_layer_volume])
  model.compile(loss='mse', optimizer='adam')

  # 모델 학습
  model.fit(x_train, [y_close, y_train], epochs=_epochs, batch_size=_batch_size)
  return model
  

# 종가(Close)와 거래량(volumes) 30개의 타임스텝으로
# 다음 5개의 종가와 거래량을 예측하시오
# ex) 01/01 ~ 01/30 데이터로 02/01~02/05 데이터 예측

df = fdr.DataReader('005930', '2000-01-01')
raw_data = df[['Close', 'Volume']].values

# 전처리
#normalize
norm_close, norm_vol, norm_concated, scaler_close, scaler_vol = normalize2(raw_data)
#x, y(close, vol) 나누기
x, y_close, y_vol = make_train_data(norm_concated)
#train, test 나누기
x_train, x_test, y_train_close, y_test_close, y_train_vol, y_test_vol = split_train_test(x, y_close, y_vol)

# 모델 생성
model = ModelLearning(x_train, y_train_close, y_train_vol, 80, 32)

# 모델 예측
output_close, output_vol = model.predict(x_test)

inv_output_close, inv_output_vol = scaler_close.inverse_transform(output_close), scaler_vol.inverse_transform(output_vol)
print("output_close: ", inv_output_close[:10])
print("output_vol: ", inv_output_vol[:10])
